In [1]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random
import copy

ROOTDIR = os.path.abspath(os.path.realpath('./')) + '/Py'

sys.path.append(os.path.join(ROOTDIR, ''))

from Stamp_transition import Stamp_transition

# Parameter

In [61]:

'''Path'''

Daily_path='./Data/Daily_Feature/'

Load_path='./Data/Processed/'

Learning_path='./Data/MCMF/'

MDP_path='./Data/CMMDP/'

'''Param'''

End_step=144

Location_list=np.load(os.path.join(Load_path,'Location_list.npy'))

date_str='2019-11-01'

'''State and Action'''

State=np.load(os.path.join(Load_path,'State.npy'))

Action=np.load(os.path.join(Load_path,'Action.npy')).item()

'''Request data'''

Request_data=pd.read_csv(os.path.join(Learning_path,'Request_data'+date_str+'.csv'))

Request_data=Request_data.drop(columns=['Unnamed: 0'])

Request_data['Dropoff_step']=Request_data.apply(lambda x:x['Dropoff_step']+1 if x['Dropoff_step']==x['Pickup_step'] else x['Dropoff_step'],axis=1)

Request_data=Request_data.loc[(Request_data['Pickup_step']<End_step)&(Request_data['Dropoff_step']<End_step)]

'''Driver data'''

Driver_data=pd.read_csv(os.path.join(Learning_path,'Driver_data'+date_str+'.csv'))

Driver_data=Driver_data.drop(columns=['Unnamed: 0'])

Driver_data=Driver_data.loc[(Driver_data['step']<End_step)]

'''Prob matrix'''

Dest_PROB_dic=np.load(os.path.join(Daily_path,'Dest_PROB_dic'+date_str+'.npy')).item()

Gain_dic=np.load(os.path.join(Daily_path,'Gain_dic'+date_str+'.npy')).item()





# Match Probability

In [62]:
def Get_prob(ratio):
        
    prob = 1- np.exp(-1*ratio)

    return prob

In [63]:
'''Order Quantity'''

Request_count=Request_data.groupby(['Pickup_Location','Pickup_step']).count()[['Order_id']]

Request_count['Transition']=Request_count.index

Request_count['Pickup_Location']=Request_count.apply(lambda x:x['Transition'][0],axis=1)

Request_count['Pickup_step']=Request_count.apply(lambda x:x['Transition'][1],axis=1)

Request_count=Request_count.rename(index=str, columns={"Order_id": "Order_Sum"})

Request_count=Request_count.reset_index(drop=True)

Request_count=Request_count[['Pickup_Location','Pickup_step','Order_Sum']]


'''Driver Quantity'''

Driver_count=Driver_data.groupby(['Location_id','step']).count()[['Driver_id']]

Driver_count['Transition']=Driver_count.index

Driver_count['Pickup_Location']=Driver_count.apply(lambda x:x['Transition'][0],axis=1)

Driver_count['Pickup_step']=Driver_count.apply(lambda x:x['Transition'][1],axis=1)

Driver_count=Driver_count.rename(index=str, columns={"Driver_id": "Driver_Sum"})

Driver_count=Driver_count.reset_index(drop=True)

Driver_count=Driver_count[['Pickup_Location','Pickup_step','Driver_Sum']]

'''Merge'''

Merge_data=Request_count.merge(Driver_count,on=['Pickup_Location','Pickup_step'],how='left')

Merge_data=Merge_data.fillna(0.01)

Merge_data['Prob']=Merge_data.apply(lambda x:Get_prob(round(x['Order_Sum']/x['Driver_Sum'],2)),axis=1)

Match_PROB_dic={}

for state in State:
    
    Match_PROB_dic[state]=0.0
    
for idx,row in Merge_data.iterrows():
    
    state=str(int(row['Pickup_Location']))+'-'+str(int(row['Pickup_step']))
    
    Match_PROB_dic[state]=row['Prob']
    
    
Match_PROB_dic   





{'127-0': 0.0,
 '127-1': 0.0,
 '127-2': 0.0,
 '127-3': 0.0,
 '127-4': 0.0,
 '127-5': 0.0,
 '127-6': 0.0,
 '127-7': 0.0,
 '127-8': 0.0,
 '127-9': 0.0,
 '127-10': 0.0,
 '127-11': 0.0,
 '127-12': 0.0,
 '127-13': 0.0,
 '127-14': 0.0,
 '127-15': 0.0,
 '127-16': 0.019801326693244747,
 '127-17': 0.0,
 '127-18': 0.0,
 '127-19': 0.0,
 '127-20': 0.0,
 '127-21': 0.0,
 '127-22': 0.0,
 '127-23': 0.0,
 '127-24': 0.0,
 '127-25': 0.0,
 '127-26': 0.0,
 '127-27': 0.0,
 '127-28': 0.0,
 '127-29': 0.0,
 '127-30': 0.0,
 '127-31': 0.0,
 '127-32': 0.0,
 '127-33': 0.0,
 '127-34': 0.0,
 '127-35': 0.0,
 '127-36': 0.019801326693244747,
 '127-37': 0.0,
 '127-38': 0.0,
 '127-39': 0.0,
 '127-40': 0.0,
 '127-41': 0.0,
 '127-42': 0.0,
 '127-43': 0.0,
 '127-44': 0.0,
 '127-45': 0.0,
 '127-46': 0.06760618009405173,
 '127-47': 0.0,
 '127-48': 0.10416586470347178,
 '127-49': 0.0,
 '127-50': 0.0,
 '127-51': 0.0,
 '127-52': 0.22119921692859512,
 '127-53': 0.0,
 '127-54': 0.0,
 '127-55': 0.22119921692859512,
 '127-56': 0.0,


# Dynamic programming

In [64]:
class Dynamic_programming(object):
    
    def __init__(self,Location_list,State,Action,End_step,Gain_dic,Match_PROB_dic,Dest_PROB_dic):
        
        '''Param'''

        self.Location_list=Location_list

        self.State=State

        self.Action=Action

        self.End_step=End_step

        self.gamma=0.8

        '''Reward'''

        self.Gain_dic=Gain_dic

        self.V_table={state:0.0 for state in self.State}

        self.Q_table={}

        for state in self.State:

            self.Q_table[state]={}

            for action in self.Action[state]:

                self.Q_table[state][action]=0.0

        '''Prob'''

        self.Match_PROB_dic=Match_PROB_dic

        self.Dest_PROB_dic=Dest_PROB_dic

        '''Iteration'''

        self.Policy={state:int(state.split('-')[0]) for state in self.State}

        self.Backwards()
        
    def Backwards(self):
        
        for t in range(self.End_step-2,-1,-1):
            
            for loc in self.Location_list:
                
                state=str(int(loc))+'-'+str(int(t))
                
                for action in self.Action[state]:
                    
                     self.Q_table[state][action]= self.Compute_Q(state,action)
                        
                self.V_table[state]=max(self.Q_table[state].values())
                
                self.Policy[state]=max(self.Q_table[state], key=self.Q_table[state].get)
                    
    def Compute_Q(self,state,action):
        
        Prob=self.Match_PROB_dic[state]
        
        step=int(state.split('-')[1])
        
        action_state=str(int(action))+'-'+str(step+1)
        
        '''Matching'''
        
        r_1=0
        
        if action_state in Dest_PROB_dic.keys():
        
            for dest_state,dest_prob in self.Dest_PROB_dic[action_state].items():

                dest_step=int(dest_state.split('-')[1])
                
                if dest_step<self.End_step:

                    r_1+=dest_prob*(self.Gain_dic[action_state][dest_state]+self.gamma**(dest_step-step)*self.V_table[dest_state])

        
        '''Not matching'''
        
        r_2=self.gamma*self.V_table[action_state]
        
        return Prob*r_2+(1-Prob)*r_1
                        
                        
                
                    
                    
                    
                    
            
            
        
        
        
        

In [66]:
Dynamic_P=Dynamic_programming(Location_list,State,Action,End_step,Gain_dic,Match_PROB_dic,Dest_PROB_dic)

np.save(MDP_path+'V_table'+date_str+'.npy',Dynamic_P.V_table)

np.save(MDP_path+'Q_table'+date_str+'.npy',Dynamic_P.Q_table)
